# GA Spiel - GA Game
This notebook implements a genetic algorithm for the problem of
hyperparameter optimization of artificial neural networks. The idea is that multiple users ("players") execute the notebook simultaneously. One player should run the "GA Master" part, the other N players perform the "GA Game" part. The game evolves over multiple generations, in each generation, each player trains a neural network and measures its performance (the development phase). The performance data and network architecture are then collected by the Master for the evolution phase. In the evolution phase, 
the master propagates the B "fittest" and R randomly chosen individuals to the next generation. The remaining N-B-R individuals are generated by cross-fertilizing randomly chosen pairs from the B+R individuals. During cross-fertilization, mutations may occur, i.e. the hyperparameters may change. This introduces the variability into the genetic algorithm. All individuals are then redistributed to the players and a new round begins.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mpievolbio-scicomp/Evolution-und-KI/blob/master/GASpiel_sc.ipynb)

In [1]:
%matplotlib inline

## The hyperparameters to optimize

In [2]:
HIDDEN_LAYER_COUNT = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
HIDDEN_LAYER_NEURONS = [8, 16, 24, 32, 64, 128, 256, 512]
HIDDEN_LAYER_RATE = [0.1, 0.2, 0.3, 0.4]
HIDDEN_LAYER_ACTIVATIONS = ['relu', 'tanh', 'sigmoid']
HIDDEN_LAYER_TYPE = ['dense', 'dropout']
MODEL_OPTIMIZER = ['rmsprop', 'sgd', 'adam']

# Define Genetic Algorithm Parameters
MAX_GENERATIONS = 100  # Max Number of Generations to Apply the Genetic Algorithm
BEST_CANDIDATES_COUNT = 3  # Number of Best Candidates to Use
RANDOM_CANDIDATES_COUNT = 1  # Number of Random Candidates (From Entire Population of Generation) to Next Population
OPTIMIZER_MUTATION_PROBABILITY = 0.1  # 10% of Probability to Apply Mutation on Optimizer Parameter
HIDDEN_LAYER_MUTATION_PROBABILITY = 0.1  # 10% of Probability to Apply Mutation o

## Code and definitions

In [3]:
from collections import OrderedDict
from contextlib import redirect_stdout, redirect_stderr
from datetime import datetime
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots
from keras import backend as K
from keras.datasets import mnist
from keras.layers import Dense, Dropout, InputLayer
from keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
from subprocess import Popen, PIPE, STDOUT
import glob
import ipywidgets as widgets
import json, io
import matplotlib as mpl
import networkx as nx
import numpy
import numpy as np
import os, shutil, shlex
import owncloud
import pandas
import pandas as pd
import random
import string
import sys
import time

In [4]:
# %load gahyparopt/gahyperopt.py
chars = string.ascii_lowercase
def random_id():
    return ''.join(random.choice(chars) for x in range(12))
 
    
class LayerLayout:

    """
    Define a Single Layer Layout
    """
    def __init__(self, layer_type):
        self.neurons = None
        self.activation = None
        self.rate = None
        self.layer_type = layer_type

class Chromosome:
    """
    Chromosome Class
    """

    def __init__(self, layer_layout, optimizer, specie, parent_a=None, parent_b=None, id=None):
        self.layer_layout = layer_layout
        self.optimizer = optimizer
#         self.result_worst = None
#         self.result_best = None
#         self.result_avg = None
#         self.result_sum = None
        self.loss = None
        self.accuracy = None
        self.specie = specie
        if id is None:
            self.id = random_id()
        else:
            self.id = id
                
        self.parent_a = parent_a
        self.parent_b = parent_b
        
        # Define Neural Network Topology
        m_model = Sequential()

        # Define Input Layer
        # m_model.add(InputLayer(input_shape=(4,)))
        m_model.add(InputLayer(input_shape=(28*28,))) # corresponding to number of pixels. 

        # Add Hidden Layers
        for layer in self.layer_layout:

            if layer.layer_type == 'dense':
                m_model.add(
                    Dense(
                        layer.neurons,
                        activation=layer.activation
                    )
                )
            elif layer.layer_type == 'dropout':
                m_model.add(
                    Dropout(rate=layer.rate)
                )

        # Define Output Layer
        # m_model.add(Dense(2, activation='sigmoid'))
        m_model.add(Dense(10, activation='softmax'))

        # Compile Neural Network
        m_model.compile(optimizer=self.optimizer,
                        loss='categorical_crossentropy',
                        metrics=['accuracy'],
                       )
        
        self.ml_model = m_model
    
    @property
    def loss(self):
        return self.__loss
    @loss.setter
    def loss(self, val):
        self.__loss = val
        
    @property
    def accuracy(self):
        return self.__accuracy
    @accuracy.setter
    def accuracy(self, val):
        self.__accuracy = val
    
    def __str__(self):
        return str(self.__dict__)
    
    def __eq__(self, other):
        return isinstance(other, Chromosome) and \
               self.id == other.id and \
               self.parent_a == other.parent_a and \
               self.parent_b == other.parent_b
          
               

    def safe_get_hidden_layer_node(self, index=0):
        """
        Return a Hidden Layer Node if Exists, Otherwise, returns None
        :param index:
        :return:
        """

        if len(self.layer_layout) > index:
            return self.layer_layout[index]

        return None

class GADriver(object):
    def __init__(self,
                 layer_counts,
                 no_neurons,
                 rates,
                 activations,
                 layer_types,
                 optimizers,
                 population_size,
                 best_candidates_count,
                 random_candidates_count,
                 optimizer_mutation_probability,
                 layer_mutation_probability,
                 ):
        
        self.layer_counts = layer_counts
        self.no_neurons = no_neurons
        self.rates = rates
        self.activations = activations
        self.layer_types = layer_types
        self.optimizers = optimizers
        self.population_size                  = population_size
        self.best_candidates_count            = best_candidates_count
        self.random_candidates_count          = random_candidates_count
        self.optimizer_mutation_probability   = optimizer_mutation_probability
        self.layer_mutation_probability       = layer_mutation_probability
        
    def generate_model_from_chromosome(self, data, chromosome):
        """
        Generate and Train Model using Chromosome Spec
        :param dataframe:
        :param chromosome:
        :return:
        """
        # Unpack data.
        x_train, y_train, x_val, y_val = data.values()

        
        # Fit Model with Data
        history = chromosome.ml_model.fit(
            x_train,
            y_train,
            epochs=10,
            steps_per_epoch=10,
    #         epochs=chromosome.number_of_epochs,
    #         steps_per_epoch=chromosome.steps_per_epoch,
            verbose=1,
            validation_data=(x_val, y_val)
        )
        
        return history

    def create_random_layer(self):
        """
        Creates a new Randomly Generated Layer
        :return:
        """

        layer_layout = LayerLayout(
            layer_type=self.layer_types[random.randint(0, len(self.layer_types) - 1)]
        )

        if layer_layout.layer_type == 'dense':
            layer_layout.neurons = self.no_neurons[random.randint(0, len(self.no_neurons) - 1)]
            layer_layout.activation = self.activations[random.randint(0, len(self.activations) - 1)]

        elif layer_layout.layer_type == 'dropout':
            layer_layout.rate = self.rates[random.randint(0, len(self.rates) - 1)]

        return layer_layout

    def generate_first_population_randomly(self):
        """
        Creates an Initial Random Population
        :return:
        """

        print("[+] Creating Initial NN Model Population Randomly: ", end='')

        result = []
        run_start = time.time()

        for current in range(self.population_size):

            # Choose Hidden Layer Count
            hidden_layer_counts = self.layer_counts[random.randint(0, len(self.layer_counts)-1)]
            hidden_layer_layout = []

            # Define Layer Structure
            for current_layer in range(hidden_layer_counts):
                hidden_layer_layout.append(self.create_random_layer())

            chromosome = Chromosome(
                layer_layout=hidden_layer_layout,
                optimizer=self.optimizers[random.randint(0, len(self.optimizers)-1)],
                specie=f"I {current}"
            )

            result.append(chromosome)

        run_stop = time.time()
        print(f"Done > Takes {run_stop-run_start} sec")

        return result

    def generate_children(self, mother: Chromosome, father: Chromosome) -> Chromosome:
        """
        Generate a New Children based Mother and Father Genomes
        :param mother: Mother Chromosome
        :param father: Father Chromosome
        :return: A new Children
        """

        # Layer Layout
        c_layer_layout = []
        layers_counts = len(mother.layer_layout) if random.randint(0, 1) == 0 else len(father.layer_layout)
        for ix in range(layers_counts):
            c_layer_layout.append(
                mother.safe_get_hidden_layer_node(ix) if random.randint(0, 1) == 0 else father.safe_get_hidden_layer_node(ix)
            )

        # Remove all Nones on Layers Layout
        c_layer_layout = [item for item in c_layer_layout if item is not None]

        # Optimizer
        c_optimizer = mother.optimizer if random.randint(0, 1) == 0 else father.optimizer

        chromosome = Chromosome(
            layer_layout=c_layer_layout,
            optimizer=c_optimizer,
            specie="",
            id=random_id(),
            parent_a=mother.id,
            parent_b=father.id,
        )
        

        return chromosome


    def mutate_chromosome(self, chromosome):
        """
        Apply Random Mutations on Chromosome
        :param chromosome: input Chromosome
        :return: Result Chromosome. May or May Not Contains a Mutation
        """

        # Apply Mutation on Optimizer
        if random.random() <= self.optimizer_mutation_probability:
            chromosome.optimizer = self.optimizers[random.randint(0, len(self.optimizers)-1)]

        # Apply Mutation on Hidden Layer Size
        if random.random() <= self.layer_mutation_probability:

            new_hl_size = self.layer_counts[random.randint(0, len(self.layer_counts)-1)]

            # Check if Need to Expand or Reduce Layer Count
            if new_hl_size > len(chromosome.layer_layout):

                # Increase Layer Count
                while len(chromosome.layer_layout) < new_hl_size:
                    chromosome.layer_layout.append(
                        self.create_random_layer()
                    )

            elif new_hl_size < len(chromosome.layer_layout):

                # Reduce Layers Count
                chromosome.layer_layout = chromosome.layer_layout[0: new_hl_size]

            else:
                pass  # Do not Change Layer Size

        return chromosome


    def evolve_population(self, population):
        """
        Evolve and Create the Next Generation of Individuals
        :param population: Current Population
        :return: A new population
        """

        print("Evolution ... ")
        # Clear Graphs from Keras e TensorFlow
        K.clear_session()
        tf.compat.v1.reset_default_graph()

        # Select N Best Candidates + Y Random Candidates. Kill the Rest of Chromosomes
        parents = []
        parents.extend(population[0:self.best_candidates_count])  # N Best Candidates
        
        print("*** Old generation ***")
        for p in population:
            print(p.id, p.parent_a, p.parent_b)
        print("*** Parents taken over ***")
        for p in parents:
            print(p.id, p.parent_a, p.parent_b)
            
        for rn in range(self.random_candidates_count):
            parents.append(population[random.randint(self.best_candidates_count, self.population_size - 1)])  # Y Random Candidate
        
        print("*** Random parents ***")
        for p in parents[self.best_candidates_count:]:
            print(p.id, p.parent_a, p.parent_b)
        
        # Create New Population Through Crossover
        new_population = []
        new_population.extend(parents)

        # Fill Population with new Random Children with Mutation
        while len(new_population) < self.population_size:
            parent_a = random.randint(0, len(parents) - 1)
            parent_b = random.randint(0, len(parents) - 1)
            while parents[parent_a].id == parents[parent_b].id:
                parent_b = random.randint(0, len(parents) - 1)
            
            new_population.append(
                self.mutate_chromosome(
                    self.generate_children(
                        mother=parents[parent_a],
                        father=parents[parent_b]
                    )
                )
            )
        
        print("*** New generation ***")
        for p in new_population:
            print(p.id, p.parent_a, p.parent_b)
        
        # Remove parents if already in previous generation
#         for i,p in enumerate(new_population):
#             if p.parent_a is None and p.parent_b is None:
#                 continue
#             for pp in parents:
#                 if p == pp:
                    
#                     print("WARNING:")
#                     print("Removing parents from {}".format(p.id))
#                     print("p:", p.id, p.parent_a, p.parent_b)
#                     print("pp:", pp.id, pp.parent_a, pp.parent_b)
#                     p.parent_a = None
#                     p.parent_b = None
                    
#                     new_population[i] = p
                    
        
#         print("*** New generation after parent cleanup***")
#         for p in new_population:
#             print(p.id, p.parent_a, p.parent_b)
        return new_population

def load_mnist():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape((60000, 28 * 28))
    x_train = x_train.astype('float32') / 255

    x_test = x_test.reshape((10000, 28 * 28))
    x_test = x_test.astype('float32') / 255

    x_val = x_train[40000:,:]
    x_train = x_train[:40000, :]

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    y_val = y_train[40000:,:]
    y_train = y_train[:40000, :]


    data = {'x_train': x_train,
            'y_train': y_train,
            'x_val': x_val,
            'y_val': y_val}
    
    return data


def generate_reference_ml(data):
    """
    Train and Generate NN Model based on https://github.com/fchollet/deep-learning-with-python-notebooks/blobs/master/2.1-a-first-look-at-a-neural-netword.ipynb'
    :param df: Dataframe to Training Process
    :return:
    """
    print("[+] Training Original NN Model: ", end='')
    run_start = time.time()

    # Define Neural model Topology
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(28 * 28,)))
    model.add(Dense(10, activation='softmax'))

    # Compile Neural model
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    # Fit Model with Data
    x_train, y_train, x_val, y_val = data.values()
    training = model.fit(x_train, y_train,
        epochs=20,
        batch_size=128,
        steps_per_epoch=300,
        verbose=0,
        validation_data=(x_val, y_val),
    )

    run_stop = time.time()
    print(f"Done > Takes {run_stop-run_start} sec")

    return model, training


def evaluate_model(ml_model, x, y, model_name="Reference Model"):
    """
    Play te Game
    :param ml_model: The model to evaluate.
    :param x: The input (test) data
    :param y: The (test) predictions.
    :return: Performance metrics (loss, accuracy).
    """
    # TODO: Implement me
    loss, accuracy = ml_model.evaluate(x, y, verbose=1)

    return loss, accuracy


In [5]:
# %load GAUtilities.py

SHAREURL='https://owncloud.gwdg.de/index.php/s/yKLtY9e230MeuRY'

sys.path.insert(0, 'gahyparopt/')

def read_chromosome(name):
    
    with open("{}.json".format(name), 'r') as jso:
        chromosome_dict = json.load(jso)
    return chromosome_from_dict(chromosome_dict)
    

def timestamp():
    dt = datetime.now()
    ts = dt.strftime("%Y%m%dT%T")
    return ts

def write_chromosome(name, chromosome):
    with open("{}.json".format(name), 'w') as jso:
        json.dump(chromosome, jso, cls=GAJSONEncoder)

class GAJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Chromosome):
            return {"layer_layout": obj.layer_layout,
                    "optimizer": obj.optimizer,
                    "loss": obj.loss,
                    "accuracy": obj.accuracy,
                    "specie": obj.specie,
                    "ml_model": get_model_str(obj.ml_model),
                    "id": obj.id,
                    "parent_a": obj.parent_a,
                    "parent_b": obj.parent_b,
                   }
        if isinstance(obj, LayerLayout):
            return {
                "neurons": obj.neurons,
                "activation": obj.activation,
                "rate": obj.rate,
                "layer_type": obj.layer_type,
            }
        if isinstance(obj, Sequential):
            pass
        if isinstance(obj, tf.python.keras.engine.sequential.Sequential):
            pass
        try:
            return json.JSONEncoder.default(self, obj)
        except:
            print(obj)
            raise

In [6]:
def layer_layout_from_dicts(layer_dicts):
    layer_layouts = []
    for dct in layer_dicts:
        layout = LayerLayout(dct['layer_type'])
        layout.neurons = dct['neurons']
        layout.activation = dct['activation']
        layout.rate = dct['rate']
        
        layer_layouts.append(layout)
    
    return layer_layouts

def chromosome_from_dict(chromosome_dict):
    layer_layout = layer_layout_from_dicts(chromosome_dict['layer_layout'])
    chromosome = Chromosome(
        layer_layout=layer_layout,
        optimizer=chromosome_dict['optimizer'],
        specie=chromosome_dict['specie'],
        id=chromosome_dict['id'],
        parent_a=chromosome_dict['parent_a'],
        parent_b=chromosome_dict['parent_b'],
    )
    chromosome.loss=chromosome_dict['loss']
    chromosome.accuracy=chromosome_dict['accuracy']
    
    return chromosome
    

def get_model_str(model):
    model_str = io.StringIO()
    with redirect_stdout(model_str):
        model.summary()
    return model_str.getvalue()


def sync_remote_to_local(name):
    """ Download all json files from the owncloud share. Overwrites all existing files. """

    public_link = SHAREURL

    # Connect to owncloud.
    oc = owncloud.Client.from_public_link(public_link)
    
    if name == 'all':
        # List content
        fhs = oc.list('.')

        # List of json files.
        jsons = [fh.get_name() for fh in fhs]
        jsons = [fh for fh in jsons if fh.split('.')[-1]=='json']

    else:
        jsons = [name+'.json']
        
    # Get all json files.
    for j in jsons:
        oc.get_file(j,j)
        
        
def sync_local_to_remote(name):
    """ Upload this players json file to the owncloud share. Overwrites all existing files on the remote side."""

    public_link = SHAREURL

    # Connect to owncloud.
    oc = owncloud.Client.from_public_link(public_link)
    
    local_file = name+".json" 
    oc.drop_file(local_file)
    
def git_pull(name):
    command = "scp -oStrictHostKeyChecking=no mplm1023@gwdu20.gwdg.de:/tmp/mplm10/{}.json .".format(name)
    if name == 'all':
        command = "scp -oStrictHostKeyChecking=no mplm1023@gwdu20.gwdg.de:/tmp/mplm10/*.json .".format(name)
    with Popen(shlex.split(command), shell=False, stdout=PIPE, stderr=STDOUT) as proc:
        print(proc.stdout.read())

def git_push(name):
    command = "scp -oStrictHostKeyChecking=no {0:s}.json mplm1023@gwdu20.gwdg.de:/tmp/mplm10/.".format(name)
    with Popen(shlex.split(command), shell=False, stdout=PIPE, stderr=STDOUT) as proc:
        print(proc.stdout.read())

def load_data():
    data = load_mnist()
    return data

def create_start_individuum(ga):
    return ga.generate_first_population_randomly()

def clear_keras_session():
    K.clear_session()
    tf.compat.v1.reset_default_graph()

def train_individuum(ga,data,individuum):
    # Reset tensorflow and keras.
    clear_keras_session()
    return ga.generate_model_from_chromosome(data, individuum)

def plot_history(history):
    hist_df = pandas.DataFrame(history.history)
    
    fig, axs = pyplot.subplots(2,1, figsize=(5,5))
    hist_df.plot(y='loss',ax=axs[0], label="Training")
    hist_df.plot(y='val_loss',ax=axs[0], label="Validation")
    axs[0].set_title("Loss")

    hist_df.plot(y='accuracy',ax=axs[1], label="Training")
    hist_df.plot(y='val_accuracy',ax=axs[1], label="Validation")
    axs[1].set_title("Accuracy")

# GA Game
## *THIS PART IS RUN BY THE N PLAYERS*

In [7]:
POPULATION_SIZE=1

In [8]:
# init
class game_instance():
    def __init__(self, ga):
        self.data = None
        self.ga = ga
        self.individuum = None
        self.name = None
        self.history = None

In [9]:
spiel = game_instance(
        ga=GADriver(
            layer_counts=HIDDEN_LAYER_COUNT,
            no_neurons=HIDDEN_LAYER_NEURONS,
            rates=HIDDEN_LAYER_RATE,
            activations=HIDDEN_LAYER_ACTIVATIONS,
            layer_types=HIDDEN_LAYER_TYPE,
            optimizers=MODEL_OPTIMIZER,
            population_size=POPULATION_SIZE,
            best_candidates_count=BEST_CANDIDATES_COUNT,
            random_candidates_count=RANDOM_CANDIDATES_COUNT,
            optimizer_mutation_probability=OPTIMIZER_MUTATION_PROBABILITY,
            layer_mutation_probability=HIDDEN_LAYER_MUTATION_PROBABILITY,
        )
    )

In [10]:
widget_dict = {'new_game':None,
               'load_data':None,
               'spieler_name':None,
               'create': None,
               'load': None,
               'train': None,
               'evaluate': None,
               'submit_evaluation':None,
               'log':None,
              }

In [11]:
log_widget = widgets.Output(layout={'border': '1px solid black', 'width':'80%', 'scroll':'true'})
widget_dict['log'] = log_widget

In [12]:
@log_widget.capture()
def log_message(widget, msg, clear=False):
    if clear:
        log_widget.clear_output()
    print("\n {} - {}\n{}\n".format(timestamp(), widget.description, msg))

In [13]:
new_game_button = widgets.Button(description="Neues Spiel")
def new_game_clicked(b):
    log_widget.clear_output()
    spiel.individuum = None
    spiel.data = None
    spiel.name = None
    spiel.history = None
    
    clear_keras_session()
    
    load_data_button.disabled=False
    spieler_name_text.disabled=False
    spieler_name_button.disabled=False
    create_button.disabled=False
    load_button.disabled=True
    train_button.disabled=True
    evaluate_button.disabled=True
    evaluation_submit_button.disabled=True
    
    log_message(b, "Neues Spiel gestartet.", clear=True)
    
    
new_game_button.on_click(new_game_clicked)
widget_dict['new_game'] = new_game_button

In [14]:
load_data_button = widgets.Button(description="Daten laden")
def load_data_clicked(b):
    spiel.data = load_data()
    load_data_button.disabled = True
    
    log_message(b, "MNIST Daten geladen.")
load_data_button.on_click(load_data_clicked)
widget_dict['load_data'] = load_data_button

In [15]:
spieler_name_text = widgets.Text(description="Spieler*in Name:")
spieler_name_button = widgets.Button(description="Spieler*in registrieren")
def spieler_name_clicked(b):
    spiel.name = spieler_name_text.value
    #spieler_name_text.disabled=True
    #spieler_name_button.disabled=True
    
    log_message(b, "{} registriert.".format(spieler_name_text.value))
    
spieler_name_button.on_click(spieler_name_clicked)    

spieler_name_widget = widgets.HBox(children=[spieler_name_text, spieler_name_button])
widget_dict['spieler_name'] = spieler_name_widget

In [16]:
create_button = widgets.Button(description="Gründe neuen Stamm")
def create_button_clicked(b):
    msg = io.StringIO()
    with redirect_stdout(msg):
        spiel.individuum = create_start_individuum(spiel.ga)[0]
        spiel.individuum.ml_model.summary()
   
    log_message(b, msg.getvalue(), True)
    
    create_button.disabled=True
    train_button.disabled=False
    
create_button.on_click(create_button_clicked)
widget_dict['create'] = create_button

In [17]:
load_button = widgets.Button(description="Neue Generation", disabled=True)

def load_button_clicked(b):
    log_message(b, "Lade neue Generation.", clear=True)
    msg = io.StringIO()
    with redirect_stdout(msg):
        clear_keras_session()
        sync_remote_to_local(spiel.name)
        spiel.individuum = read_chromosome(spiel.name)
        spiel.individuum.ml_model.summary()
    log_message(b, msg.getvalue())
    
    load_button.disabled=True
    train_button.disabled=False
    
load_button.on_click(load_button_clicked)
widget_dict['load'] = load_button

In [18]:
train_button = widgets.Button(description = "Individuum entwickeln", disabled=True)
def train_button_clicked(b):
    log_message(b, "Training startet.", True)
    with log_widget:
        try:
            history = train_individuum(spiel.ga,spiel.data,spiel.individuum)
            log_message(b, "Training beendet.", True)
            plot_history(history)
            show_inline_matplotlib_plots()

        except:
            print("WARNING: Training failed, accuracy set to 0, loss set to 100.")
            history = None
    
    
    train_button.disabled=True
    evaluate_button.disabled=False
    
train_button.on_click(train_button_clicked)
    
widget_dict['train'] = train_button

In [19]:
evaluation_value_text = widgets.FloatText(description="Evaluation", disabled=True)
evaluation_submit_button = widgets.Button(description="Jetzt mitteilen", disabled=True)
submit_widget = widgets.HBox(children=[evaluation_value_text, evaluation_submit_button ])
evaluate_button = widgets.Button(description="Individuum evaluieren", disabled=True)

def evaluate_button_clicked(b):
    log_message(b, "Evaluation startet")
    with log_widget:
        try:
            loss, accuracy = evaluate_model(spiel.individuum.ml_model, spiel.data['x_val'], y=spiel.data['y_val'])
        except:
            accuracy = 0.0
            loss = 100.0
    
    evaluate_button.disabled=True
        
    spiel.individuum.loss = loss
    spiel.individuum.accuracy = accuracy
    evaluation_value_text.value = accuracy
    evaluation_submit_button.disabled=False
    
evaluate_button.on_click(evaluate_button_clicked)

widget_dict['evaluate'] = evaluate_button

def evaluation_submit_button_clicked(b):
    log_widget.clear_output()
    with log_widget:
        write_chromosome(spiel.name, spiel.individuum)
        sync_local_to_remote(spiel.name)
    evaluation_submit_button.disabled=True
    load_button.disabled=False
    msg = "Evaluation übermittelt für Spieler*in {}.".format(spiel.name)
    log_message(b, msg)
    
evaluation_submit_button.on_click(evaluation_submit_button_clicked) 

widget_dict['submit_evaluation'] = submit_widget



In [20]:
preparation_widgets = [widgets.HTML(value="<h3>Neues Spiel starten</3>")]


preparation_widgets += [widget_dict[k] for k in ["new_game",
                                                 "load_data",
                                                 "spieler_name",
                                                 "create"]]

preparation_box = widgets.VBox(children=preparation_widgets)


separator = widgets.HTML(value="<hr>")

development_widgets = [widgets.HTML(value="<h3>Entwicklungsphase</3>")]
development_widgets += [widget_dict[k] for k in ["load",
                                                 "train",
                                                 "evaluate",
                                                 "submit_evaluation",
                                                 "log"]]
development_box = widgets.VBox(children=development_widgets)           
gui = widgets.VBox(children=[preparation_box, separator, development_box])
    


In [21]:
display(gui)

## Spiel startet
Clicke zum Spielstart in diese Zelle und wähle im Menu *Run -> Run all above selected cell* aus.  
In colab: *Ctr-+F8*. Dadurch baut sich die Spielkonsole auf. In dieser nacheinander:
* **Neues Spiel**
* **Daten laden**
* Spieler\*innamen eintragen (ohne Leer- und Sonderzeichen), **Spieler\*in registrien**
* **Gründe neuen Stamm**

## In jeder Generation:
* **Individuum entwickeln**
* **Individuum evaluieren**
* **Jetzt mitteilen**

An dieser Stelle übernimmt der Master den Evolutionsschritt und erzeugt eine neue Generation.
Nach Verteilung der Individuen der neuen Generation an die Spieler (**Neue Generation**) geht es,
wie oben beschrieben, weiter.

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# GA Master
## *ONLY THE MASTER PLAYER SHOULD RUN THIS PART*

In [22]:
def load_population():
    json_fnames = glob.glob("*.json")
    
    population = []
    names = []
    for j in json_fnames:
        name = j.split(".")[0]
        names.append(name)
        individuum = read_chromosome(j.split('.')[0])
        population.append(individuum)
        
    population = OrderedDict(zip(names, population))
    population=OrderedDict(sorted(population.items(),key=lambda x: x[1].accuracy, reverse=True))
    return population

In [23]:
class Model(object):
    generation_performance = {}
    population=None
    generations={}
    index=0

In [24]:
sync_remote_to_local('all')

In [ ]:
model = Model()
model.population = load_population()

TOTAL_POPULATION_SIZE = len(model.population)

In [ ]:
ga=GADriver(
            layer_counts=HIDDEN_LAYER_COUNT,
            no_neurons=HIDDEN_LAYER_NEURONS,
            rates=HIDDEN_LAYER_RATE,
            activations=HIDDEN_LAYER_ACTIVATIONS,
            layer_types=HIDDEN_LAYER_TYPE,
            optimizers=MODEL_OPTIMIZER,
            population_size=TOTAL_POPULATION_SIZE,
            best_candidates_count=BEST_CANDIDATES_COUNT,
            random_candidates_count=RANDOM_CANDIDATES_COUNT,
            optimizer_mutation_probability=OPTIMIZER_MUTATION_PROBABILITY,
            layer_mutation_probability=HIDDEN_LAYER_MUTATION_PROBABILITY,
        )

In [ ]:
cmap = pyplot.cm.nipy_spectral  # define the colormap
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# force the first color entry to be grey

# create the new map
segmented_cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, TOTAL_POPULATION_SIZE)

segmented_cmap_list = [segmented_cmap(i) for i in range(segmented_cmap.N)]

# define the bins and normalize
bounds = np.linspace(0, TOTAL_POPULATION_SIZE, TOTAL_POPULATION_SIZE+1)
norm = mpl.colors.BoundaryNorm(bounds, TOTAL_POPULATION_SIZE)

players = model.population.keys()
color_player_dict = dict(zip(segmented_cmap_list, players))
player_color_dict = dict(zip(players, segmented_cmap_list))
# Add "0" for 0 generation
player_color_dict['0'] = "k"

In [ ]:
def get_accuracy_plot(df):
    fig, ax = pyplot.subplots(2,1,sharex=True,figsize=(7,8))
    mn = df.accuracy.min()*0.9
    mx = min(df.accuracy.max()*1.1, 1.0)
    bar_width = 0.8
    df.iloc[::-1,::-1].plot.barh(x='name',
                                 y='accuracy',
                                 ax=ax[0],
                                 grid=True,
                                 color=color_player_dict
                                 )
    df['accuracy'].plot.hist(ax=ax[1],
                             grid=True,
                             bins=TOTAL_POPULATION_SIZE,
                             range=(mn,mx),
                             rwidth=bar_width,
                            )
    ax[1].set_xlim([mn,mx])
   
    return fig, ax


In [ ]:
# Pull button
pull_button = widgets.Button(description="Generation aktualisieren")
def pull_button_clicked(b):
    log_widget.clear_output()
    with log_widget:
        sync_remote_to_local('all')
        
pull_button.on_click(pull_button_clicked)
#################################################################
#
# Generation count
generation_count = widgets.BoundedIntText(value=0, min=0, max=0)

def on_generation_count_change(change):
    log_widget.clear_output()
    with log_widget:
        df = model.generation_performance[change['new']]
        
    plot_accuracy_vs_name(df)
    plot_generation_tree()
    
generation_count.observe(on_generation_count_change, names=['value'])    
#################################################################
#
# Generation load button
generation_button = widgets.Button(description="Generation laden")

def generation_button_clicked(b):
    log_widget.clear_output()
    with log_widget:
        model.population = load_population()

        df = pandas.DataFrame([
            {'name': key,
             'loss': model.population[key].loss,
             'accuracy': model.population[key].accuracy
            } for key in model.population.keys()
        ])
        
        model.index = model.index + 1
        model.generation_performance.update({model.index: df})
        model.generations.update({model.index: model.population})
        generation_count.max = model.index
        generation_count.min = 1
        generation_count.value = model.index
    
generation_button.on_click(generation_button_clicked)
#################################################################
#
# Evolve button
evolve_button = widgets.Button(description="Evolution!")
def evolve_button_clicked(b):
    log_widget.clear_output()
    with log_widget:
        new_chromosomes = ga.evolve_population(list(model.population.values()))
        model.population = OrderedDict(zip(model.population.keys(), new_chromosomes))
        
evolve_button.on_click(evolve_button_clicked)
#################################################################
#
# Distribute button
distribute_button = widgets.Button(description="Generation verteilen")

def distribute_button_clicked(b):
    log_widget.clear_output()
    with log_widget:
        for name,individuum in model.population.items():
            write_chromosome(name=name,chromosome=individuum)
            sync_local_to_remote(name)
            
distribute_button.on_click(distribute_button_clicked)
#################################################################
#
# Plot widgets
accuracy_plot_widget = widgets.Output(layout={'border': '1px solid black', 'width':'50%', 'scroll':'true'})
generation_tree_widget = widgets.Output(layout={'border': '1px solid black', 'width':'50%', 'scroll':'true'})
log_widget = widgets.Output(layout={'border': '1px solid black', 'width':'80%', 'scroll':'true'})

def plot_accuracy_vs_name(df):
    accuracy_plot_widget.clear_output()
    with accuracy_plot_widget:
        
        fig, ax = get_accuracy_plot(df)

        show_inline_matplotlib_plots()
        
def plot_generation_tree():
    generation_tree_widget.clear_output()
    with generation_tree_widget:
        fig, ax = pyplot.subplots(1,1, figsize=(7,8))
        graph, pos = get_tree(model, generation_count.value)
        color = [player_color_dict[data["player"]] for v, data in graph.nodes(data=True)]
        nx.draw(graph, pos, ax, node_color=color)       
        show_inline_matplotlib_plots()
##################################################################
#
# GUI
gui = widgets.VBox(children=[
    widgets.HBox([pull_button, generation_button]),
    generation_count,
    widgets.HBox([
        accuracy_plot_widget,
        generation_tree_widget,
    ]),
    evolve_button,
    distribute_button,
    log_widget,
])

In [ ]:
# Build graph
def get_tree(model, max_generation):
    
    graph = nx.DiGraph()
    graph.add_node("X-0", layer=0, player="0")
    
    for g,generation in model.generations.items():
        
        if g > max_generation:
            break
        for player, chrm in generation.items():
            graph.add_node("{}-{}".format(chrm.id, g), layer=g, player=player)


    edges = []

    for g,generation in model.generations.items():
        if g > max_generation:
            break
            
        for chrm in generation.values():
            parent_a = chrm.parent_a
            parent_b = chrm.parent_b
            id = chrm.id

            if parent_a is None and parent_b is None:
                parent_a = id
                parent_b = id

            if g == 1:
                parent_a = "X"
                parent_b = "X"

            parent_a = "{}-{}".format(parent_a, g-1) 
            parent_b = "{}-{}".format(parent_b, g-1) 

            if parent_a not in graph.nodes:
                parent_a = "{}-{}".format(id, g-1)
            if parent_b not in graph.nodes:
                parent_b = "{}-{}".format(id, g-1)

            id = "{}-{}".format(id, g)

            edges.append((parent_a, id))
            edges.append((parent_b, id))


    graph.add_edges_from(edges)

    nodes_to_remove = []
    for node,data in graph.nodes(data=True):
        if "layer" not in data.keys():
            nodes_to_remove.append(node)

    assert len(nodes_to_remove) == 0

    pos = nx.multipartite_layout(graph, subset_key="layer")

    return graph,pos

In [ ]:
display(gui)